In [1]:
import os
def eachFile(filepath,l=[]):
    pathDir=os.listdir(filepath)
    for allDir in pathDir:
        child = os.path.join(filepath, allDir)
        if os.path.isdir(child):
            eachFile(child)
        elif allDir=='sop.xlsx':
            l.append(child)
    return l

In [2]:
import pandas as pd
def chemical_data(filepath):
    sop=pd.read_excel(filepath,sheetname='sop',header=None)
    df=pd.DataFrame(sop)
    chemical_result=pd.DataFrame()
    for i in range(0,len(sop)):
        d1=df.iloc[i][0]
        if d1=='chemical':
            chemical=pd.DataFrame(df.iloc[i,1:]).T
            chemical_result=chemical_result.append(chemical)    
    return chemical_result

In [3]:
def dna_data(filepath):
     sop=pd.read_excel(filepath,sheetname='sop',header=None)
     df=pd.DataFrame(sop)
     dna_result=pd.DataFrame()
     for i in range(0,len(sop)):
        d1=df.iloc[i][0]
        if d1=='dna':
            dna=pd.DataFrame(df.iloc[i,1:]).T
            dna_result=dna_result.append(dna)
     return dna_result

In [4]:
def wash_data(filepath):
     sop=pd.read_excel(filepath,sheetname='sop',header=None)
     df=pd.DataFrame(sop)
     wash_result=pd.DataFrame()
     for i in range(0,len(sop)):
        d1=df.iloc[i][0]
        if d1=='wash':
            wash=pd.DataFrame(df.iloc[i,1:]).T
            wash_result=wash_result.append(wash)
     return wash_result

In [5]:
def sop_data(filepath):
     sop=pd.read_excel(filepath,sheetname='sop',header=None)
     df=pd.DataFrame(sop)
     sop_result=pd.DataFrame()
     for i in range(0,len(sop)):
        d1=df.iloc[i][0]
        if d1=='Experiment Purpose':
            sop_result=sop_result.join(pd.DataFrame(df.iloc[i+1]).T,how='outer')
        if d1=='equipment':
            t=pd.DataFrame(df.iloc[i,1:]).T
            t.index=[1]
            sop_result=sop_result.join(t,how='outer',lsuffix='t')
        if d1=='method_step':
            t=df.iloc[i+1:,1:3]
            for a in range(0,len(t)):
                a=pd.DataFrame(t.iloc[a]).T
                a.index=[1]
                sop_result=sop_result.join(a,how='outer',lsuffix='a')
     return sop_result

In [6]:
filePathC ='C:\\Users\\Administrator\\Desktop\\data\\'
import pymssql
l=eachFile(filePathC)
for i in l:
   x=chemical_data(i)
   conn=pymssql.connect(host='localhost',user='test',password='abc123...',database='t',charset='UTF-8')
   cur=conn.cursor()
   for x in range(0,len(i)):
       cur.execute("insert into dna values(i.iloc[x].tolist()[0],i.iloc[x].tolist()[1],i.iloc[x].tolist()[3])")
       conn.commit()
   conn.close

NameError: name 'pymssql' is not defined